## Preamble

In [ ]:
import pandas as pd
import numpy as np
import logging
from collections import defaultdict
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
import matplotlib.ticker as ticker
pd.options.display.max_rows = 30
logger = logging.getLogger()

import datetime as dt
import matplotlib.dates as mdates


In [ ]:
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

## Reading and processing the "r" file

In [ ]:
r = pd.read_excel('./r.xlsx', skiprows=[0])
r.columns = ['date', 'r']
ins1 = SimpleExpSmoothing(r['r']).fit(smoothing_level=0.05,optimized=False)
r['r_smoothed'] = ins1.fittedvalues
display(r)

## Reading and processing the number of vaccinated

In [ ]:
df_vax = pd.read_excel('./vax.xlsx', skiprows=[0])
df_vax.columns = ['date', 'vax1', 'vax2', 'vax3']
df_vax['daily_gain'] = df_vax['vax2'].diff()
df_vax['daily_gain3'] = df_vax['vax3'].diff()
display(df_vax)


## Reading and processing the numebr of daily cases

In [ ]:
df_cases = pd.read_excel('./cases.xlsx', skiprows=[0])
df_cases.columns = ['date', 'age_group', 'vax3_pos', 'vax2_pos', 'vax0_pos', 'cases_vax3_norm', 'cases_vax2_norm', 'cases_vax0_norm']
df_cases = df_cases[df_cases['age_group'] == 'כלל האוכלוסיה']
display(df_cases)


## Calculating the crude VE using the daily cases

In [ ]:
def crude_ve(df_cases):
    df_cases['ve_dose2'] = 1 - (df_cases['cases_vax2_norm'] / df_cases['cases_vax0_norm'])
    df_cases['ve_dose3'] = 1 - (df_cases['cases_vax3_norm'] / df_cases['cases_vax0_norm'])
    df_cases['ve_dose3_2'] = 1 - (df_cases['cases_vax3_norm'] / df_cases['cases_vax2_norm'])
    
    return df_cases

df_cases = crude_ve(df_cases)
display(df_cases)

## merging all files together

In [ ]:
df_all = pd.merge(df_vax, df_cases, "inner", on='date')
df_all = pd.merge(df_all, r, "left", on='date') #left merge to avoid cutting new dates before R calculation
df_all['date'] = pd.to_datetime(df_all['date'], format='%d-%m-%Y')
display(df_all)

## Plotting the VE of dose 2 for santity check

In [ ]:
fig, axs = plt.subplots(nrows = 1, ncols = 1, figsize = (7,5))

data = df_all['ve_dose2']
ins1 = SimpleExpSmoothing(data).fit(smoothing_level=0.05,optimized=False)
smoothed = ins1.fittedvalues
axs.plot(df_all['date'], smoothed, linewidth=5)
axs.xaxis.set_major_locator(ticker.MultipleLocator(14))
axs.set_ylabel(r'Crude VE (smoothed)', fontsize=20)


for tick in axs.get_xticklabels():
    tick.set_rotation(90)
    
axs.grid()


## The main figures

In [ ]:
data = df_all['ve_dose3']
ins1 = SimpleExpSmoothing(data).fit(smoothing_level=0.05,optimized=False)
smoothed_ve3 = ins1.fittedvalues

data = df_all['ve_dose2']
ins1 = SimpleExpSmoothing(data).fit(smoothing_level=0.05,optimized=False)
smoothed_ve2 = ins1.fittedvalues


fig, ax1 = plt.subplots(nrows = 1, ncols = 1, figsize = (7,5))
isr_protection = \
                1 - \
                 (smoothed_ve2 * (df_all['vax2'] - df_all['vax3']) + \
                 smoothed_ve3 * df_all['vax3']) / (9.4 * 10 ** 6)

ax1.plot(df_all['date'], isr_protection, color = 'black',  linewidth = 3)
ax1.set_xlabel('Date')
ax1.set_ylabel('Fraction of susceptible population', color='black', fontsize = 14)
ax1.grid()

In [ ]:
data = df_all['ve_dose3']
ins1 = SimpleExpSmoothing(data).fit(smoothing_level=0.05,optimized=False)
smoothed_ve3 = ins1.fittedvalues

data = df_all['ve_dose2']
ins1 = SimpleExpSmoothing(data).fit(smoothing_level=0.05,optimized=False)
smoothed_ve2 = ins1.fittedvalues


isr_protection = \
                1 - \
                 (smoothed_ve2 * (df_all['vax2'] - df_all['vax3']) + \
                 smoothed_ve3 * df_all['vax3']) / (9.4 * 10 ** 6)


fig, ax1 = plt.subplots(nrows = 1, ncols = 1, figsize = (7,5))
ax2 = ax1.twinx()

ax1.plot(df_all['date'], isr_protection, color = 'black', linewidth = 5)
ax2.plot(df_all['date'], df_all['r_smoothed'], color = 'red', linewidth = 3)

ax1.set_xlabel('Date')
ax1.set_ylabel('Fraction of susceptible population', color='black', fontsize = 14)
ax2.set_ylabel('R(smoothed)', color='red', fontsize = 20)

y0, y1 = ax1.get_ylim()
x = dt.datetime(2021, 2, 15)
ax1.vlines(x, y0, y1, linestyle = 'dashed', color = 'blue', linewidth = 0.75)
ax1.set_ylim(y0,y1)
ax1.text(x, y1*0.81, 'End of third lockdown', color = 'blue', rotation = 'vertical')

y0, y1 = ax1.get_ylim()
x = dt.datetime(2021, 6, 15)
ax1.vlines(x, y0, y1, linestyle = 'dashed', color = 'blue', linewidth = 0.75)
ax1.set_ylim(y0,y1)
ax1.text(x, y1*0.7, 'End of restrictions', color = 'blue', rotation = 'vertical')


y0, y1 = ax1.get_ylim()
x = dt.datetime(2021, 7, 29)
ax1.vlines(x, y0, y1, linestyle = 'dashed', color = 'blue', linewidth = 0.75)
ax1.set_ylim(y0,y1)
ax1.text(x, y1*0.7, 'Green passport', color = 'blue', rotation = 'vertical')




#ax1.grid()

[t.set_color('red') for t in ax2.yaxis.get_ticklines()]
[t.set_color('red') for t in ax2.yaxis.get_ticklabels()]





plt.show()










